# Estimating the mean

Recall that if $X_1,\ldots,X_n\sim N(\mu,\sigma^2)$ are independent, then $\dfrac{\overline X - \mu}{S/\sqrt n} \sim t(n - 1)$, i.e., it has the *Student's $t$ distribution* with $n - 1$ *degrees of freedom*.

Suppose we obtained a random sample with $n = 16$, $\overline x = 5$, and $s^2 = 9$.  What's the probability that we obtained this sample if $\mu = 5$?  In particular, we would like to find a constant $a$ for which $P(|\overline X - 5|<a) = 0.95$.

In class, we reduced this problem to $P\left(T > \frac{4a}{3}\right) = 0.025$.

## Quantile functions

First, a quick definition of *quantile function*, which I only briefly stated in class.  The quantile function is the inverse of cumulative distribution function.  For a given continuous random variable $X$, we define $Q_X$ so that for any $p\in(0,1)$, $P(X\leq Q_X(p)) = p$.  (The definition is slightly more technical than this, but this will work for us.)  For example, $Q_X(0.5)$ is the median of $X$.

To find values of the quantile function for a probability distribution in R, we use a function starting with **q**.  For Student's _t_ distribution, we use **qt**.  The first argument is the desired probability $p$, the second argument is the parameter (degrees of freedom) of the distribution, and we can provide the **lower.tail** option to tell us that we're really looking for the area in the right tail of the distribution, i.e., to compute $Q_X(1 - p)$ instead of $Q_X(p)$.

In [3]:
qt(0.025, 15, lower.tail=FALSE)

[1] 2.13145

This gave us $a = \frac{3}{4}\cdot 2.1314 = 1.5986$.

Now let's check our work with a Monte Carlo simulation.  We can generate random variates of the a normal distribution with **rnorm**.  The first argument will be the number of variates to generate, the second will be the mean, and the third the standard deviation.  (We don't actually know $\sigma$, but it's reasonable to use $\sigma\approx s = 3$.)

In [2]:
rnorm(16, 5, 3)

[1]  8.763297  2.510814  7.173856  3.754719  6.485343  1.485419  2.873843
 [8]  2.748800  6.024509 10.806966  6.965050  4.446912  7.308774  4.067470
[15]  7.762867  6.194802

We're curious how far away our sample mean is from the assumed population mean of 5, so we're going to compute the following lots of times.

In [3]:
abs(mean(rnorm(16, 5, 3)) - 5)

[1] 1.87199

In [4]:
N <- 10000
mean(replicate(N, abs(mean(rnorm(16, 5, 3)) - 5) < 1.5986))

[1] 0.9649

We were expecting 0.95.  Not bad!

# Estimating variance

This is the example I stated poorly in class.  Let's try again.

Same example as above.  Let $n = 16$, $\overline x = 5$, and $s^2 = 9$.  We would like to find two constants $b$ and $c$ such that $P(b < S^2 < c)=0.95$ if $\sigma^2 = 9$.  An in particular, for which $P(S^2 < b) = P(S^2 > c) = \frac{1-0.95}{2}=0.025$, i.e., the areas in the left and right tails should agree.

(Note that we're *not* subtracting 9 like I did in class.  I confused the mean and variance cases.)

Using Student's theorem, we know that $\frac{(n-1)S^2}{\sigma^2}=\frac{15 S^2}{9}\sim\chi^2(15)$.

So we have the following.  First, let's find $b$:

$$P(S^2 < b) = 0.025$$
$$P\left(\frac{15 S^2}{9} < \frac{15 b}{9}\right) = 0.025$$

Letting $X=\frac{15 S^2}{9}\sim\chi^2(15)$, we  have:

$$P\left(X < \frac{15b}{9}\right) = 0.025$$
$$P\left(X > \frac{15b}{9}\right) = 1 - 0.025 = 0.975$$

Now let's use R to figure out what $\frac{15b}{9}$ must be.  We'll use the **qchisq** function, which works just like **qt**.

In [6]:
qchisq(0.975, 15, lower.tail = FALSE)

[1] 6.262138

So we have:

$$\frac{15b}{9} = 6.2621$$
$$b = \frac{9}{15}\cdot 6.2621 = 3.7573$$

Now let's play the same game for $c$:

$$P(S^2 > c) = 0.025$$
$$P\left(\frac{15 S^2}{9} > \frac{15 c}{9}\right) = 0.025$$
$$P\left(X > \frac{15 c}{9}\right) = 0.025$$

In [7]:
qchisq(0.025, 15, lower.tail = FALSE)

[1] 27.48839

$$\frac{15 c}{9} = 27.4884$$
$$c = \frac{9}{15}\cdot 27.4884 = 16.4930$$

Consequently, we expect that $P(3.7573<S^2 < 16.4930) = 0.95$.

Let's check with a Monte Carlo simulation.  This time we don't know $\mu$, but we do know that $\mu\approx\overline x = 5$.

We can't enter a three-part inequality directly in R, but we can combine two regular inequalities using **&&** for "and".  The following will check if the variance of a random sample from our distribution lies in the desired interval.  Note that we compute the sample variance first and save it to a variable so we don't compute two different sample variances.

In [15]:
s2 <- var(rnorm(16, 5, 3)); 3.7573 < s2 && s2 < 16.4930

[1] TRUE

Now let's run our Monte Carlo simulation.

In [16]:
N <- 10000
mean(replicate(N, {s2 <- var(rnorm(16, 5, 3)); 3.7573 < s2 && s2 < 16.4930}))

[1] 0.9485

Very nice!